Instalamos las librerias no standares de Python

In [3]:
!pip install factor-analyzer
!pip install import_ipynb
!pip install ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 516.7 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 30.9 MB/s eta 0:00:00
  Created wheel for factor-analyzer: filename=factor_analyzer-0.5.0-py2.py3-none-any.whl size=42487 sha256=3dd18df93287495c51728a92bf3b7a7aadffc4d97b9728dafa516a120338f676
  Stored in directory: /root/.cache/pip/wheels/74/a2/6c/26fb1addf1ce6c60a8cef8397f2999f0a1e6e2fcddc8abf33e
Successfully built factor-analyzer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00


Importamos las librerias de Python

In [4]:
import random

import numpy as np
import pandas as pd

from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

from scipy.optimize import minimize
from scipy.special import expit as sigmoid  # Sigmoid function is the logistic function

from sklearn.metrics import roc_auc_score

Importamos las librerias auxiliares desde otro Colab

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%run '/content/drive/My Drive/Colab Notebooks/Funciones_Raras_1.ipynb'
%run '/content/drive/My Drive/Colab Notebooks/Funciones_Raras_2.ipynb'

Funciones que se van a usar luego en el programa:

- Normalización
- Generación aleatoria de datos

In [7]:
def normalize_matrix(matrix):
    # Convert the matrix to a NumPy array for easier manipulation
    np_matrix = np.array(matrix)

    # Find the minimum and maximum values in the matrix
    min_val = np_matrix.min()
    max_val = np_matrix.max()

    # Perform the normalization
    normalized_matrix = (np_matrix - min_val) / (max_val - min_val)

    return normalized_matrix

def normalize_list(float_list):
    min_val = min(float_list)
    max_val = max(float_list)
    range_val = max_val - min_val
    if range_val == 0:
        # Avoid division by zero if all values are the same
        return [0 for _ in float_list]
    else:
        # Perform the normalization
        normalized_list = [(x - min_val) / range_val for x in float_list]
        return normalized_list


def generate_random_matrix(rows, columns):
    # Generate the matrix of random floats
    matrix = [[random.random() for _ in range(columns)] for _ in range(rows)]

    return matrix

def generate_random_list(rows):
    # Generate the list of random floats
    random_list = [random.random() for _ in range(rows)]

    return random_list


# Validación cruzada para un ancho de banda h determinado.
def cv_score(h, Z, Y):
    n = len(Y)
    betas_i = [Local_logit(Z.drop(i), Y.drop(i), Z.iloc[[i]], h, 'gauss') for i in range(n)]
    beta0_i = [beta[0] for beta in betas_i]
    cv = np.sum((Y - sigmoid(beta0_i))**2) / n
    return cv

Aquí voy a ir desglosando el código poco a poco para comprobar que se hace lo mismo que en el código original.

Primero, la entrada de datos. Aquí voy a generar una matriz aleatoria de datos y comprobamos el código debería funcionar primero, y luego utilizamos los datos originales.

```
# - Código R - #

## hay que normalizar los datos:
xdatos0<-scale(xdatos) #################!!!!!!!!!

### mis datos:
misdatos<-cbind(xdatos0,Y)
```



In [8]:
USAR_ALEATORIA = False

# Variables con la entrada de datos
matrix_datos = None
salida_datos = None

if(USAR_ALEATORIA):

  # Generar la matriz de entrada normalizada
  rows    = 10
  columns = 5

  matrix_datos = generate_random_matrix(rows, columns)
  salida_datos = generate_random_list(rows)

else:

  # Usar la matriz de datos del fichero

  # -- Dónde está el directorio principal
  PROJECT_FOLDER = "/content/drive/My Drive/Colab Notebooks/"
  # -- Hay que usar sensor.csv de 120MB
  DATA_FILE      = "/datos/sensor.csv"

  # Cárgamos el archivo en un dataframe
  # NOTA: La columna para el sesonr 15 está vacía, esto es correcto
  data = pd.read_csv( PROJECT_FOLDER + DATA_FILE, sep = ",", header=0)

  # Eliminamos las dos primeras columnas que son la fecha y el índice
  data = data.drop(data.columns[[0]], axis=1) # axis = 1 significa columna
  data = data.drop(data.columns[[0]], axis=1)

  # Cambiamos los estados originales de Y de esta manera:
  #      {NORMAL, BROKEN, RECOVERING}
  #      {1     , 0     , 0}
  data['new'] = 1
  for i in range(0,data.shape[0]):
    if ( (data.iat[i, data.shape[1]-2] ) != "NORMAL"):
      data.iat[i, data.shape[1]-1] = 0

  # Borramos la columna de Y antigua
  data = data.drop(data.columns[[  data.shape[1]-2  ]], axis=1)

  # Comprobamos el estado de salud de los datos
  data.isna().sum()

  # Esto nos dice que las columnas 15 y 50 son una patata, las quitamos
  # directamente.
  dataTemp = data.copy()
  del dataTemp['sensor_15']
  del dataTemp['sensor_50']

  # Buscamos fila por fila y quitamos las que tengan NAs
  deleteTheseRows = [False]*dataTemp.shape[0]
  for i in range(0, dataTemp.shape[0]):

    if (   dataTemp.iloc[i,].isna().sum()  >= 1   ) :
      deleteTheseRows[i] = True

  print("Total eliminadas ", sum(deleteTheseRows))

  #Aquí tenemos todos los datos originales sin filas NA
  dataCurrent = dataTemp.drop(dataTemp[deleteTheseRows].index)

  # Separammos los datos y definimos X e Y
  X = dataCurrent.iloc[:, 0:dataCurrent.shape[1]-1]
  Y = dataCurrent.iloc[:,   dataCurrent.shape[1]-1]
  matrix_datos = X
  salida_datos = Y


matrix_datos = normalize_matrix(matrix_datos)
salida_datos = normalize_list(salida_datos)

print("Datos - X:")
print(matrix_datos)
print("Salida - Y:")
print(salida_datos)

# Convertir en un panda dataframe para luego
matrix_DF       = pd.DataFrame(matrix_datos)
salida_datos_DF = pd.Series(salida_datos)

Total eliminadas  24505
Datos - X:
[[0.0012327  0.02354601 0.0266059  ... 0.07899305 0.03385417 0.10069445]
 [0.0012327  0.02354601 0.0266059  ... 0.07899305 0.03385417 0.10069445]
 [0.00122237 0.02367622 0.0266059  ... 0.0779803  0.03356482 0.10185185]
 ...
 [0.00119826 0.02384982 0.02526041 ... 0.1086516  0.07769095 0.1160301 ]
 [0.00120318 0.02384982 0.02526042 ... 0.1112558  0.0769676  0.1170428 ]
 [0.00119826 0.02384982 0.02526042 ... 0.1137153  0.0752315  0.1170428 ]]
Salida - Y:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

```
2. Análisis factorial:

p.prima<-fa.parallel(xdatos0,fa='fa',fm='mle')$nfact
af0<-fa(mat.cor,nfactors=p.prima,rotate='varimax',fm='mle')
p<-ncol(xdatos0)
n<-nrow(xdatos0)

### scores:
L.mat<-matrix(af0$loadings,p,p.prima)
G<-L.mat;U<-scale(xdatos0)
Z<-U%*%G # esta es la matriz de scores
```


In [10]:
# Bartlett's test de esfericidad y Kaiser-Meyer-Olkin (KMO) test
chi_square_value, p_value = calculate_bartlett_sphericity(matrix_DF)
kmo_all, kmo_model = calculate_kmo(matrix_DF)

# Análisis factorial
fa = FactorAnalyzer(rotation=None, method='principal', is_corr_matrix=False)
fa.fit(matrix_DF)
ev, v = fa.get_eigenvalues()

# Guardamos el eigenvector
actual_eigenvalues = ev

# Análisis paralelo con datos aleatorios
PASOS_BOOTSSTRAP = 10
random_eigenvalues = []
for _ in range(PASOS_BOOTSSTRAP):

    # Imprimimos por donde vamos cada 10% de las veces
    if (i+1) % (PASOS_BOOTSSTRAP // 10) == 0:
        percentage = (i + 1) / PASOS_BOOTSSTRAP * 100
        print(f"{int(percentage)}%")

    # Calculamos los datos aleatorios para el modelo
    random_data = np.random.rand(matrix_DF.shape[0], matrix_DF.shape[1])

    # Hacemos el factor
    fa_random = FactorAnalyzer(rotation=None, method='principal', is_corr_matrix=False)
    fa_random.fit(random_data)
    ev_random, v_random = fa_random.get_eigenvalues()
    random_eigenvalues.append(ev_random)

# Calcular la media de los eigenvalores
mean_random_eigenvalues = np.mean(random_eigenvalues, axis=0)

# Determinar cuantos factores nos quedamos
n_factors_to_retain = sum(actual_eigenvalues > mean_random_eigenvalues)

print(f"Nos quedamos con estos factores: {n_factors_to_retain}")

/usr/local/lib/python3.10/dist-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


Nos quedamos con estos factores: 8


In [11]:
# Calculamos la matrix de correlación de Pearson
# (esto está escondido en el código original)
mat_cor = matrix_DF.corr()


# Inicializar el objeto de análisis factorial con el número de factores y el
# método de rotación
#
# ---- 'varimax', que es un método de rotación ortogonal que tiene como objetivo
#      maximizar la varianza de las cargas al cuadrado de un factor en todas las
#      variables.
#
# ---- 'mle' (ml), el método de extracción factorial para la estimación de máxima
#      verosimilitud.

fa = FactorAnalyzer(n_factors = n_factors_to_retain,
                    rotation='varimax', method='ml')

# ----------------
# IMPORTANTE:
# ----------------
# Aquí uso la factorización del eje principal (method='principal'),
# que es menos sensible a las violaciones de la normalidad.
# Si no, los datos aleatorios no convergen luego con el método del eigenvector.
#
# Esto puede pasar porque no hay suficientes datos, muchos factores
# (en el caso aleatorio 2, osea que no) o multicolinealidad en los datos.
# fa = FactorAnalyzer(n_factors = n_factors_to_retain,
#                    rotation='varimax', method='principal')
# ----------------

# Ajustar el modelo de análisis factorial a la matriz de correlación
fa.fit(mat_cor)

# La carga de factores, comunalidades y todo lo demás se puede acceder
# directamente desde el objeto
loadings = fa.loadings_
communalities = fa.get_communalities()
unique_variances = fa.get_uniquenesses()

print("Factor Loadings:")
print(loadings)
print("\nCommunalities:")
print(communalities)
print("\nUnique Variances:")
print(unique_variances)

/usr/local/lib/python3.10/dist-packages/factor_analyzer/factor_analyzer.py:443: RuntimeWarning: invalid value encountered in log
  error = -(np.sum(np.log(values) - values) - n_factors + corr_mtx.shape[0])


Factor Loadings:
[[-4.22190873e-01  8.86672064e-01  1.90856419e-02 -6.09916511e-02
  -6.79083415e-02 -6.97415903e-02  1.64606448e-02 -8.74101518e-02]
 [-3.68625678e-01  4.31739640e-01  1.42626501e-01 -2.93810653e-03
   8.70868349e-02  2.44914343e-02  7.13809860e-01  8.97488330e-03]
 [-6.53574665e-01  6.45567728e-01  1.49353108e-02 -6.41624603e-02
  -4.46256526e-02 -5.08494148e-02  2.90819325e-01 -3.76026328e-02]
 [-5.83977610e-01  5.38507990e-01  3.57110412e-01  8.13067005e-02
  -2.52441615e-02 -2.48337138e-03  2.38598195e-01  1.57734094e-01]
 [-4.02082980e-01  8.88528249e-01  1.20857495e-02 -6.78912767e-02
  -7.39781155e-02 -9.11107379e-02  2.41715143e-02 -9.64911431e-02]
 [-4.65608084e-01  7.84574684e-01 -2.17206013e-01 -1.44541906e-01
  -1.43298012e-01 -1.11353385e-01  7.80622255e-02 -7.24800964e-02]
 [-2.77684647e-01  9.22827492e-01  2.19273775e-01  6.30582730e-02
   9.67369657e-03  2.26500079e-02  7.53739664e-02  4.90709850e-02]
 [-4.25816142e-01  8.32855865e-01  1.62271287e-02  5

/usr/local/lib/python3.10/dist-packages/factor_analyzer/factor_analyzer.py:574: UserWarning: Failed to converge: ABNORMAL_TERMINATION_IN_LNSRCH
  warnings.warn(f"Failed to converge: {res.message}")


In [12]:
# ----------------
# IMPORTANTE:
# ----------------
#
# Hay que decidir cual de estos dos métodos es el correcto.
# Por ahora me quedo con el factor_scores_df.

# Calcular los scores de los factores para cada observación
factor_scores    = fa.transform(matrix_DF)
factor_scores_df = pd.DataFrame(factor_scores, columns=[f'Factor{i+1}' for i in range(n_factors_to_retain)])
print(factor_scores_df)

# Esto es diferente a como sale en el código de R, que se hace multiplicando
# las matrices:
result = matrix_DF.dot(loadings)
print(result)

         Factor1   Factor2   Factor3   Factor4   Factor5   Factor6   Factor7  \
0      -0.655955 -0.704382 -0.444334 -0.983929 -0.180842 -0.929551 -1.326319   
1      -0.655955 -0.704382 -0.444334 -0.983929 -0.180842 -0.929551 -1.326319   
2      -0.637052 -0.702821 -0.441832 -0.982092 -0.185389 -0.944568 -1.283860   
3      -0.637465 -0.706529 -0.436206 -0.983684 -0.188655 -0.955752 -1.280319   
4      -0.643317 -0.699674 -0.460845 -0.969094 -0.172720 -0.934133 -1.299186   
...          ...       ...       ...       ...       ...       ...       ...   
195810 -0.470516 -0.653687 -0.130269 -0.928236 -0.460941 -0.813203 -1.226172   
195811 -0.477759 -0.657890 -0.130243 -0.929577 -0.459431 -0.821887 -1.230435   
195812 -0.476979 -0.658077 -0.127659 -0.921726 -0.462085 -0.829175 -1.231096   
195813 -0.476529 -0.656719 -0.123092 -0.914094 -0.466834 -0.837556 -1.224164   
195814 -0.474154 -0.653655 -0.128844 -0.911289 -0.463657 -0.836008 -1.228503   

         Factor8  
0      -0.026283  
1

```
### para reescalar las beta:
v.mean<-apply(xdatos0,2,mean);
#mat.mean<-matrix(v.mean,n,p,byrow=T)
v.sd<-apply(xdatos0,2,sd)
mat.sd<-diag((v.sd)^(-1))

scale.betas<-mat.sd%*%G
```

In [13]:
# Esto creo que intenta escalar las cargas factoriales por la inversa de
# las desviaciones estándar de las variables originales

mean = matrix_DF.mean(axis=0)
sd   = matrix_DF.std(axis=0)

# Crea una matriz diagonal con la inversa de las desviaciones estándar
mat_sd = np.diag(1 / sd)

# Realizar la multiplicación de matrices para escalar las cargas factoriales
scale_betas = np.dot(mat_sd, loadings)
scale_betas_df = pd.DataFrame(scale_betas, index= matrix_DF.columns,
                              columns=[f'Factor{i+1}' for i in range(n_factors_to_retain)])

print(scale_betas_df)

        Factor1      Factor2     Factor3     Factor4     Factor5     Factor6  \
0  -2271.945552  4771.468974  102.706008 -328.215789 -365.436735 -375.302040   
1   -299.086023   350.293805  115.720622   -2.383845   70.658276   19.871230   
2   -540.781387   534.156280   12.357789  -53.089366  -36.924201  -42.073872   
3   -661.037196   609.567567  404.233418   92.035640  -28.575290   -2.811068   
4    -11.146104    24.630808    0.335028   -1.882008   -2.050740   -2.525672   
5    -74.797596   126.037976  -34.893053  -23.219930  -23.020105  -17.888361   
6   -371.075741  1233.193474  293.020083   84.266075   12.927161   30.267674   
7   -601.453928  1176.386665   22.920386   70.732504   -7.345057   68.192454   
8   -579.633254  1203.792870   -3.649068   83.149783   -7.833487  101.277170   
9   -611.718712  1192.227720 -112.035217   66.625289  -35.391678  110.799902   
10   -64.791619   236.952371   -7.462933    3.017400  -13.411204   16.007900   
11   -71.652400   187.375536   66.398066

```
##3.  Local-LR:
source('Funciones_datos.R')
source('MeyerMIR_1.R')

## 3.1. selecciono ancho de banda con CV:
rango.h<-diff(range(Z)); hmin<-rango.h/n;hmax<-rango.h;
h.grid<-seq(hmin,hmax,length=50)
n<-nrow(Z);#p.prima<-ncol(Z)

n.h<-length(h.grid)
```

In [ ]:


# Calculamos el rango de los scores
rango_h = factor_scores_df.max().max() - factor_scores_df.min().min()

# Calculamos el ancho de banda máximo (underfitting) y mínimo (overfitting)
# y generamos los valores intermedios

H_INTERMEDIOS = 50
hmin   = rango_h / rows
hmax   = rango_h
h_grid = np.linspace(hmin, hmax, num = H_INTERMEDIOS)

# Mostrar los hs
# print(h_grid)


[ 2.07165655  2.4521649   2.83267325  3.21318159  3.59368994  3.97419829
  4.35470663  4.73521498  5.11572333  5.49623167  5.87674002  6.25724837
  6.63775671  7.01826506  7.39877341  7.77928175  8.1597901   8.54029845
  8.92080679  9.30131514  9.68182349 10.06233183 10.44284018 10.82334853
 11.20385687 11.58436522 11.96487357 12.34538191 12.72589026 13.10639861
 13.48690695 13.8674153  14.24792365 14.62843199 15.00894034 15.38944869
 15.76995703 16.15046538 16.53097373 16.91148207 17.29199042 17.67249877
 18.05300711 18.43351546 18.81402381 19.19453215 19.5750405  19.95554885
 20.33605719 20.71656554]
50


```
cv.score<-function(h)
{
  betas.i<-sapply(1:n,function(i){return(Local.logit(x.mat=Z[-i,],y=Y[-i],x0.mat=Z[i,],h=h,K='gauss'))})
  beta0.i<-betas.i[1,]
  cv<-sum((Y-plogis(beta0.i))^2,na.rm=T)/n
  return(cv)
}

cv.values<-sapply(h.grid,cv.score)
ind.cv<-which.min(cv.values)
hcv<-h.grid[ind.cv]
```         

In [ ]:
cv_values = np.array([cv_score(h_grid, factor_scores_df, salida_datos_DF) for h in h_grid])
ind_cv = np.argmin(cv_values)
hcv = h_grid[ind_cv]

print("Ancho de banda optimo:", hcv)

    Factor1    Factor2
1 -2.042501  17.634269
2 -2.392574  17.322328
3 -1.291649  16.056607
4 -2.258478  17.780569
5 -2.792639  15.489628
6 -2.817172  17.899393
7 -1.481023  16.767033
8 -1.643401  16.131673
9 -1.795789  17.815416
    Factor1    Factor2
0 -2.720324  16.707116
1
9
[    Factor1   Factor2
1  0.677823  0.927153
2  0.327750  0.615212
3  1.428675 -0.650509
4  0.461846  1.073453
5 -0.072315 -1.217488
6 -0.096848  1.192277
7  1.239301  0.059917
8  1.076923 -0.575443
9  0.924535  1.108300]
[ 2.07165655  2.4521649   2.83267325  3.21318159  3.59368994  3.97419829
  4.35470663  4.73521498  5.11572333  5.49623167  5.87674002  6.25724837
  6.63775671  7.01826506  7.39877341  7.77928175  8.1597901   8.54029845
  8.92080679  9.30131514  9.68182349 10.06233183 10.44284018 10.82334853
 11.20385687 11.58436522 11.96487357 12.34538191 12.72589026 13.10639861
 13.48690695 13.8674153  14.24792365 14.62843199 15.00894034 15.38944869
 15.76995703 16.15046538 16.53097373 16.91148207 17.29199042

ValueError: ignored

```
### Modelo optimo:
local.fit<-Local.logit(x.mat=Z,y=Y,x0.mat=Z,h=hcv,K='gauss')
beta0<-local.fit[,1]
```



In [ ]:
local_fit = Local_logit(x_mat=Z, y=Y, x0_mat=Z, h=hcv, K='gauss')
beta0 = local_fit.iloc[:, 0]

```
#3.2. Definir beta_gorro y estimar la fiabilidad del sistema.

#### el modelo local:
#### para cada r?plica proporciona una matriz de betas de dimensiones n x (p+1)
#### en el caso param?trico lo que calculamos es un vector de dimensi?n 1 x (p+1)
#### el coeficiente beta_j es el mismo para todos las filas de la muestra !!!
# no voy a calcular las beta.x porque no lo estamos guardando ahora mismo!!!
# cuando quiera valorar importancia de las componentes lo guardaremos
## de momento nos conformamos con la estimaci?n de la fiabilidad que nos la da el vector de beta0


estim.prob<-plogis(beta0)#Fiabilidad en cada punto

estim.mat<-cbind(xdatos0,estim.prob)
estim<-estim.prob
```

In [ ]:
# Esta línea aplica la función logística (función sigmoide) a los valores de
# beta0. En R calcula la función de distribución acumulativa de la distribución
# logística que es equivalente a la función sigmoide. El resultado es un vector
# de probabilidades estimadas (estim.prob) correspondiente a la fiabilidad en
# cada punto.
estim_prob = sigmoid(beta0)

# Matriz de estimaciones contiene los datos originales más una columna
# adiccional que son las estimaciones de la línea anterior
estim_mat = pd.concat([xdatos0, pd.Series(estim_prob, name='estim_prob')], axis=1)

```
#3.3. Accuracy: average square error

iauc.LL<-NA#auc(y=Y,prob=iestim)##!!!!
auc.LL<-auc(y=Y,prob=estim)


In [ ]:
# Y es una matriz con etiquetas binarias verdaderas y estim_prob  es una matriz
# que contiene las probabilidades predichas de la función logistica
auc_LL = roc_auc_score(Y, estim_prob)

```
##4. GLOBAL FIT: predictions and sensitivity analysis

mdat.g<-cbind(Z,y=Y)
global.fit<-mon.lik(mdat=mdat.g)


### Hacemos la transformaci?n hacia atr?s para recuperar el modelo en los datos originales
####  hago esto para ver los coeficientes beta del modelo sobre los datos originales y
####  comprobar si el m?todo conserva la monoton?a:
## pongo comentarios en lo siguiente porque ya se ha calculado para el modelo local aunque all? no se utiliz?:


betas.z<-global.fit$g.beta ## estos son los coeficientes beta.tilde!!! sobre las Z, no sobre las X
v.mean<-apply(xdatos0,2,mean);

#mat.mean<-matrix(v.mean,n,p,byrow=T)
beta0.g<-betas.z[1]-t(v.mean)%*%scale.betas%*%betas.z[-1]  ## beta0<-b0-mean(x)%*%Sigma^{-1}%*%G%*%b_(0)
betas.g<-scale.betas%*%betas.z[-1]
betas.g<-c(beta0.g,betas.g)

g.prob<-global.fit$prob

auc.g<-auc(y=Y,prob=g.prob)
```

In [ ]:
# Combinar Z e Y en la dirección de las columnas
mdat_g = np.column_stack((Z, Y))

# Ajustar el modelo global
# Esta línea llama a una función mon.lik, función para ajustar un modelo de
# probabilidad monótono, con los datos combinados mdat.g
global_fit = mon_lik(mdat=mdat_g)

# Extraer los coeficientes de estimación
betas_z = global_fit['g_beta']

# Calcular la media de los datos originales
v_mean = xdatos0.mean(axis=0)

# Calcular la intercepción
# Esta línea calcula la intersección (beta0.g) para el modelo global ajustando.
# El primer término son los coeficientes de estimación.
# El segundo término es una multiplicación de matrices:
#     La primera matriz es una matriz de medias
#     La segunda matriz es también una multiplicación de matrices
#         La primera matriz son el scale_betas que calculamos antes
#         La segunda matriz son los coeficientes restantes de Betas_Z
beta0_g = betas_z[0] - np.dot(v_mean.T, np.dot(scale_betas, betas_z[1:]))

# Escalar los coeficientes
betas_g = np.dot(scale_betas, betas_z[1:])

# Combinar los coeficientes con la intercepción
betas_g = np.concatenate(([beta0_g], betas_g))

# Extraer las probabilidades
g_prob = global_fit['prob']

# Calcular la AUC
auc_g = roc_auc_score(Y, g_prob)

```
resultado<-matrix(c(auc.LL,iauc.LL,auc.g),1,3)

colnames(resultado)<-c('auc.LL','iauc.LL','auc.g')

betas<-matrix(round(betas.g,3),1,p+1)
colnames(betas)<-c('beta0',colnames(misdatos[,-(p+1)]))

beta0.ll<-local.fit[,1]
betas.ll<-local.fit[,-1]%*%t(scale.betas)

betas.ll<-cbind(beta0.ll,betas.ll)
m.betas<-colMeans(betas.ll)
plot(m.betas[-1])
abline(h=0)

### Birnbaum final:
estim<-plogis(beta0.ll)
importancia<-betas.ll[,-1]*estim*(1-estim)
plot(importancia)
birnbaum<-colMeans(importancia)
```